In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [14]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2))
        
        self.fc = torch.nn.Linear(7*7*64, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [15]:
model = CNN().to(device)

In [16]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.209965736
[Epoch:    2] cost = 0.0608952902
[Epoch:    3] cost = 0.0455710888
[Epoch:    4] cost = 0.0368914753
[Epoch:    5] cost = 0.0312152058
[Epoch:    6] cost = 0.0260432139
[Epoch:    7] cost = 0.0224106759
[Epoch:    8] cost = 0.0184271038
[Epoch:    9] cost = 0.0148821911
[Epoch:   10] cost = 0.0137344887
[Epoch:   11] cost = 0.0118853068
[Epoch:   12] cost = 0.00981370173
[Epoch:   13] cost = 0.00842928328
[Epoch:   14] cost = 0.00750073558
[Epoch:   15] cost = 0.00561115518
Learning Finished!
